In [1]:
from requests_html import HTMLSession, HTML
from selenium import webdriver
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


/opt/conda/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#Kiểm tra xem dữ liệu có hợp pháp không
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.worldweatheronline.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')

True

In [3]:
session = HTMLSession()

In [4]:
url = "https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx"

In [5]:
#Tên file chứa data
#Start 2009
filename='weather_data'

In [6]:
#tập hợp các loại thời tiết (để phân loại và chuyển về 1 số loại nhất định)
list_w = set()

In [7]:
def parse_data (url,filename,year):
    name = f"{filename}_{year}.csv"
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    browser  = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    # Khai báo một biến đợi với thời gian tối đa là 10 giây
    wait = WebDriverWait(browser, 5)
    #browser = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver')
    browser.get(url)
    with open(name,'w') as f:
         f.write('time\tday\tmonth\ttemperature\tforecast\twind\tcloud\train\tpressure\tweather\n')

    for y in year:
        if ((y%4 == 0)|(y%400==0)):
            ferb_day=30
        else:
            ferb_day = 29
        #danh sách ngày trong tháng (để tìm kiếm ngày trong web)
        #thêm 0 để chỉ số bằng số tháng
        day = [[0],
                  range(1,32),
                  range(1,ferb_day),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,32),
                  range(1,31),
                  range(1,32),
                  range(1,31),
                  range(1,32)]    
        for m in range(12): #Lay data 1 thang test
            m +=1
            if ((m==12)&(y==2024)):
                break;#Chỉ lấy đến tháng 11 năm 2016
            for d in day[m]:
                if (d<10):
                    d = f"0{d}"
                if (m<10):
                    m_str = f"0{m}"
                    text = f"{m_str}{d}{y}"
                else:
                    text = f"{m}{d}{y}"
                #Xác định vị trí của textbox và điền giá trị vào
                #textbox = wait.until(EC.presence_of_element_located((By.XPATH, "//INPUT[@id='ctl00_MainContentHolder_txtPastDate']")))

                textbox = browser.find_element(by=By.XPATH, value="//INPUT[@id='ctl00_MainContentHolder_txtPastDate']")
                #time.sleep(2)  # Đợi 2 giây

                textbox.click()
                textbox.send_keys(text)
                #Bấm nút để lấy dữ liệu
                button = browser.find_element(by=By.XPATH, value="//INPUT[@id='ctl00_MainContentHolder_butShowPastWeather']")

                button.click()
                time.sleep(1)#Nghỉ 1 giây để trang có thời gian load
                r = HTML(html=browser.page_source)
               
                soup = BeautifulSoup(browser.page_source)
                table = soup.find('table', class_='days-details-table')
                if table is not None:
                        rows = table.find_all('tr')
                        if rows is not None:
                            timeOut = []
                            tempOut = []

                            weatherOut = []
                            forcastOut = []
                            rainOut = []
                            rainPercentOut = []
                            pressureOut = []
                            cloudOut = []
                            windOut = []
                            windGustOut = []
                            for i in range(2, 10):  # Lặp qua các phần tử từ 2 đến 11 (tổng cộng 10 lần)
                                row = rows[i].find_all('td')
                                #times =re.findall("\d{2}:\d{2}", row.text)

                                times = row[0].find('p', class_='days-comment').text
                                temp = row[0].find('p', class_='days-temp').text
                                weather = row[1].find('img', class_='days-table-forecast-img').get('alt')
                                rain = row[3].text
                                rainPercent = row[5].text
                                forecast = row[2].find('p', class_='days-table-forecast-p').text
                                cloud = row[5].text
                                pressure = row[6].text
                                wind = row[7].text
                                windGust = row[8].text

                                if not times or  times == "" or times is None:
                                         times = 0
                                elif temp is None or not temp  or temp == "":
                                         temp = 0 
                                elif weather is None or not weather or weather == "":
                                         weather = Unknow
                                elif rain is None or not rain or rain == "":
                                         rain = 0 
                                elif rainPercent is None or not rainPercent or rainPercent == "":
                                         rainPercent = 0 
                                elif forecast is None or not forecast or forecast == "":
                                         forecast = 0 
                                elif cloud is None or not cloud or cloud == "":
                                         cloud = 0 
                                elif pressure is None or not pressure or pressure == "":
                                         pressure = 0 
                                elif wind is None or not wind or wind == "":
                                         wind = 0 
                                elif windGust is None or not windGust or windGust == "":
                                         windGust = 0 

                                timeOut.append(times)
                                tempOut.append(temp)
                                weatherOut.append(weather)
                                rainOut.append(rain)
                                forcastOut.append(forecast)
                                cloudOut.append(cloud)
                                pressureOut.append(pressure)
                                windOut.append(wind)



                            print(timeOut)
                            print(d)
                            print(m)
                            print(tempOut)
                            print(weatherOut)
                            print(rainOut)
                            print(forcastOut)
                            print(cloudOut)
                            print(pressureOut)
                            print(windOut)


                            with open(name,"a") as f:
                                 for i in range(8):
                                    f.write(f"{timeOut[i]}\t{d}\t{m}\t{tempOut[i]}\t{forcastOut[i]}\t{windOut[i]}\t{cloudOut[i]}\t{rainOut[i]}\t{pressureOut[i]}\t{weatherOut[i]}\n")

In [8]:
parse_data(url,filename,[2012])#nam 2016 chỉ lấy đến hết tháng 11


['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
01
1
['24 °c', '23 °c', '24 °c', '27 °c', '30 °c', '30 °c', '27 °c', '25 °c']
['Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Patchy rain possible', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.2mm', '0.0mm', '0.0mm', '0.0mm']
['26 °c', '26 °c', '26 °c', '30 °c', '34 °c', '34 °c', '30 °c', '28 °c']
['33%', '53%', '41%', '36%', '73%', '28%', '26%', '31%']
['1013mb', '1012mb', '1013mb', '1014mb', '1011mb', '1010mb', '1011mb', '1012mb']
['7 km/h', '9 km/h', '10 km/h', '8 km/h', '4 km/h', '7 km/h', '5 km/h', '9 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
02
1
['24 °c', '24 °c', '24 °c', '27 °c', '31 °c', '31 °c', '27 °c', '24 °c']
['Partly cloudy', 'Clear', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Partly cloudy', 'Clear', 'Partly cloudy']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
14
1
['25 °c', '24 °c', '24 °c', '29 °c', '33 °c', '31 °c', '27 °c', '26 °c']
['Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['27 °c', '26 °c', '26 °c', '31 °c', '36 °c', '35 °c', '30 °c', '28 °c']
['41%', '29%', '58%', '28%', '39%', '28%', '29%', '45%']
['1010mb', '1009mb', '1009mb', '1009mb', '1007mb', '1007mb', '1007mb', '1009mb']
['8 km/h', '7 km/h', '7 km/h', '9 km/h', '12 km/h', '13 km/h', '13 km/h', '11 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
15
1
['25 °c', '24 °c', '24 °c', '30 °c', '32 °c', '31 °c', '26 °c', '25 °c']
['Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Clear', 'Partly cloudy']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm'

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
27
1
['25 °c', '24 °c', '25 °c', '28 °c', '32 °c', '32 °c', '28 °c', '26 °c']
['Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['27 °c', '26 °c', '27 °c', '31 °c', '35 °c', '35 °c', '30 °c', '28 °c']
['32%', '34%', '28%', '24%', '40%', '25%', '14%', '17%']
['1010mb', '1009mb', '1010mb', '1010mb', '1009mb', '1008mb', '1009mb', '1011mb']
['3 km/h', '1 km/h', '7 km/h', '2 km/h', '2 km/h', '3 km/h', '10 km/h', '5 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
28
1
['25 °c', '24 °c', '24 °c', '28 °c', '29 °c', '29 °c', '26 °c', '25 °c']
['Clear', 'Clear', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Overcast', 'Overcast', 'Patchy rain possible']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.2mm']
['28 °c', '26 °c', '26 °c', 

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
10
2
['25 °c', '24 °c', '25 °c', '29 °c', '34 °c', '33 °c', '28 °c', '25 °c']
['Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Partly cloudy', 'Clear', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['27 °c', '26 °c', '27 °c', '32 °c', '38 °c', '36 °c', '30 °c', '27 °c']
['15%', '12%', '9%', '6%', '11%', '33%', '19%', '20%']
['1012mb', '1011mb', '1011mb', '1012mb', '1010mb', '1009mb', '1010mb', '1012mb']
['8 km/h', '5 km/h', '7 km/h', '7 km/h', '6 km/h', '2 km/h', '10 km/h', '12 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
11
2
['25 °c', '25 °c', '24 °c', '29 °c', '35 °c', '33 °c', '28 °c', '26 °c']
['Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Partly cloudy', 'Clear', 'Partly cloudy']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['27 °c', '27 °c', '26 °c', '31 °c', '38 °c', '35 °c', '30 °c', '28 °c']
['17%', '11%', '1

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
23
2
['25 °c', '24 °c', '24 °c', '29 °c', '33 °c', '32 °c', '27 °c', '26 °c']
['Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['28 °c', '27 °c', '27 °c', '33 °c', '38 °c', '36 °c', '30 °c', '28 °c']
['10%', '21%', '22%', '5%', '5%', '9%', '9%', '13%']
['1008mb', '1007mb', '1008mb', '1009mb', '1007mb', '1005mb', '1006mb', '1008mb']
['8 km/h', '9 km/h', '9 km/h', '7 km/h', '10 km/h', '17 km/h', '12 km/h', '9 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
24
2
['25 °c', '24 °c', '25 °c', '30 °c', '34 °c', '33 °c', '28 °c', '26 °c']
['Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Clear', 'Clear', 'Partly cloudy', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['27 °c', '27 °c', '27 °c', '34 °c', '39 °c', '38 °c', '32 °c', '28 °c']
['34%', '4

...

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
25
10
['25 °c', '25 °c', '25 °c', '30 °c', '32 °c', '31 °c', '27 °c', '26 °c']
['Clear', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Clear', 'Partly cloudy', 'Partly cloudy']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['28 °c', '28 °c', '28 °c', '35 °c', '38 °c', '38 °c', '30 °c', '29 °c']
['22%', '41%', '37%', '37%', '25%', '20%', '29%', '34%']
['1009mb', '1008mb', '1009mb', '1009mb', '1007mb', '1005mb', '1007mb', '1008mb']
['4 km/h', '1 km/h', '4 km/h', '4 km/h', '2 km/h', '7 km/h', '8 km/h', '7 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
26
10
['25 °c', '24 °c', '25 °c', '30 °c', '33 °c', '34 °c', '28 °c', '25 °c']
['Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Clear', 'Clear', 'Clear', 'Partly cloudy']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['28 °c', '27 °c', '28 °c', '35 °c', '38

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
07
11
['26 °c', '25 °c', '25 °c', '29 °c', '32 °c', '32 °c', '27 °c', '26 °c']
['Clear', 'Partly cloudy', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Clear', 'Clear', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['29 °c', '28 °c', '28 °c', '34 °c', '38 °c', '40 °c', '31 °c', '29 °c']
['21%', '28%', '36%', '28%', '11%', '9%', '13%', '13%']
['1009mb', '1009mb', '1010mb', '1010mb', '1008mb', '1007mb', '1008mb', '1010mb']
['5 km/h', '3 km/h', '5 km/h', '4 km/h', '1 km/h', '5 km/h', '7 km/h', '6 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
08
11
['25 °c', '25 °c', '26 °c', '30 °c', '32 °c', '32 °c', '27 °c', '26 °c']
['Clear', 'Partly cloudy', 'Clear', 'Clear', 'Clear', 'Patchy rain possible', 'Patchy rain possible', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.1mm', '0.1mm', '0.0mm']
['28 °c', '27 °c', '29 °c', '35 °c', '38 °c', '39 °

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
20
11
['26 °c', '25 °c', '26 °c', '29 °c', '31 °c', '30 °c', '27 °c', '26 °c']
['Partly cloudy', 'Partly cloudy', 'Clear', 'Clear', 'Patchy rain possible', 'Patchy rain possible', 'Clear', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.2mm', '0.2mm', '0.0mm', '0.0mm']
['29 °c', '28 °c', '30 °c', '34 °c', '36 °c', '34 °c', '31 °c', '29 °c']
['28%', '32%', '24%', '19%', '88%', '79%', '13%', '19%']
['1011mb', '1010mb', '1011mb', '1011mb', '1010mb', '1008mb', '1010mb', '1011mb']
['5 km/h', '5 km/h', '6 km/h', '6 km/h', '8 km/h', '10 km/h', '9 km/h', '8 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
21
11
['25 °c', '25 °c', '26 °c', '29 °c', '31 °c', '29 °c', '27 °c', '26 °c']
['Clear', 'Partly cloudy', 'Partly cloudy', 'Patchy rain possible', 'Patchy rain possible', 'Moderate or heavy rain shower', 'Moderate or heavy rain shower', 'Partly cloudy']
['0.0mm', '0.0mm', '0.0mm', '0.1mm',

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
04
12
['25 °c', '25 °c', '25 °c', '29 °c', '31 °c', '31 °c', '27 °c', '26 °c']
['Partly cloudy', 'Cloudy', 'Cloudy', 'Partly cloudy', 'Patchy rain possible', 'Patchy rain possible', 'Partly cloudy', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.2mm', '0.2mm', '0.0mm', '0.0mm']
['27 °c', '27 °c', '27 °c', '33 °c', '35 °c', '36 °c', '30 °c', '29 °c']
['31%', '68%', '71%', '43%', '88%', '72%', '28%', '4%']
['1010mb', '1010mb', '1011mb', '1012mb', '1009mb', '1007mb', '1009mb', '1010mb']
['7 km/h', '5 km/h', '4 km/h', '4 km/h', '5 km/h', '1 km/h', '5 km/h', '9 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
05
12
['25 °c', '24 °c', '25 °c', '29 °c', '33 °c', '33 °c', '27 °c', '26 °c']
['Partly cloudy', 'Partly cloudy', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['27 °c', '26 °c', '27 °c', '32 °c', '39 

['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
18
12
['25 °c', '24 °c', '24 °c', '30 °c', '34 °c', '33 °c', '28 °c', '27 °c']
['Clear', 'Partly cloudy', 'Clear', 'Clear', 'Clear', 'Cloudy', 'Partly cloudy', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['27 °c', '26 °c', '26 °c', '33 °c', '40 °c', '38 °c', '32 °c', '30 °c']
['10%', '33%', '4%', '4%', '6%', '70%', '33%', '15%']
['1010mb', '1010mb', '1011mb', '1012mb', '1009mb', '1009mb', '1010mb', '1011mb']
['9 km/h', '8 km/h', '11 km/h', '11 km/h', '6 km/h', '3 km/h', '5 km/h', '9 km/h']
['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
19
12
['26 °c', '26 °c', '25 °c', '30 °c', '33 °c', '34 °c', '29 °c', '27 °c']
['Clear', 'Partly cloudy', 'Partly cloudy', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear']
['0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm', '0.0mm']
['29 °c', '28 °c', '28 °c', '33 °c', '37 °c', '39 °c', '32 °c', '30 °c']
['18%',